In [1]:
import datetime
import pandas as pd
import pyaurorax

aurorax = pyaurorax.PyAuroraX()

# AuroraX ML-enhanced searching

Integrated with the AuroraX search engine, there is the ability to search through ephemeris and conjunctions based on various machine learning model metadata that has been uploaded to the platform. 

Two use cases to highlight, but there are many more: 
  - finding all conjunctions between spacecrafts and the THEMIS ASIs where an ML model believes it is not cloudy in the THEMIS ASI
  - retrieve all 1-minute times where an ML model believes THEMIS ASIs have Amorphous Pulsating Aurora (APA) in the field of view.

More information about the available ML metadata can be found [here](https://docs.aurorax.space/ml/overview).

# Conjunction Searching

Let's first have a look at how to do conjunction searches, and find results by leveraging some ML-derived metadata.

## UCalgary Amorphous Pulsating Aurora (APA) model

UCalgary has developed a machine learning model for identifying Amorphous Pulsating Aurora (APA) in the THEMIS all-sky imagers. This is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Metadata fields in AuroraX can be searched upon, allowing users to filter results based on them. Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where this particular model thinks there is APA in the field-of-view for that camera, over a 1 month period.

More information about this model can be found [here](https://docs.aurorax.space/ml/models/ucalgary_apa/).

In [2]:
# set timeframe, distance, and conjunction type
start = datetime.datetime(2008, 1, 1, 0, 0, 0)
end = datetime.datetime(2008, 1, 31, 23, 59, 59)
conjunction_types = [aurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# set ground criteria block
ground = [
    aurorax.search.GroundCriteriaBlock(
        programs=["themis-asi"],
        metadata_filters=aurorax.search.MetadataFilter(expressions=[
            # only find records that were classified as APA
            aurorax.search.MetadataFilterExpression("calgary_apa_ml_v1", "classified as APA", operator="="),

            # with a confidence of at least 95%
            aurorax.search.MetadataFilterExpression("calgary_apa_ml_v1_confidence", 95, operator=">="),
        ]))
]

# set space criteria block
space = [aurorax.search.SpaceCriteriaBlock(programs=["themis"], hemisphere=["northern"])]

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground,
    space=space,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2025-02-27 08:59:03.104667] Search object created
[2025-02-27 08:59:03.129508] Request submitted
[2025-02-27 08:59:03.129534] Request ID: f63c9bda-985c-4fc6-8ebd-bb8e21c3a756
[2025-02-27 08:59:03.129538] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/f63c9bda-985c-4fc6-8ebd-bb8e21c3a756
[2025-02-27 08:59:03.129541] Waiting for data ...
[2025-02-27 08:59:05.337728] Checking for data ...
[2025-02-27 08:59:05.777991] Data is now available
[2025-02-27 08:59:05.778072] Retrieving data ...
[2025-02-27 08:59:06.273276] Retrieved 52.8 kB of data containing 8 records


In [3]:
# show the conjunction results
#
# NOTE: while here we format the results into a Pandas dataframe, this
# is not required. We actually don't include Pandas as a dependency since
# it's used simply as a nice add-on to view data. If you're good with slicing
# and dicing lists and dictionaries, you'll be fine without it.
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
0,nbtrace,2008-01-01 11:30:00,2008-01-01 11:36:00,"[DataSource(identifier=45, program='themis-asi...",383.962128,428.502480,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-01 11:30:00,2008-01-01 11:36:00
1,nbtrace,2008-01-01 11:30:00,2008-01-01 11:59:00,"[DataSource(identifier=45, program='themis-asi...",340.029413,499.355723,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-01 11:30:00,2008-01-01 11:59:00
5,nbtrace,2008-01-08 05:50:00,2008-01-08 05:52:00,"[DataSource(identifier=49, program='themis-asi...",485.213758,499.938201,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-08 05:50:00,2008-01-08 05:52:00
3,nbtrace,2008-01-14 04:50:00,2008-01-14 04:59:00,"[DataSource(identifier=49, program='themis-asi...",150.330448,200.986916,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-14 04:50:00,2008-01-14 04:59:00
6,nbtrace,2008-01-14 04:50:00,2008-01-14 04:59:00,"[DataSource(identifier=49, program='themis-asi...",203.965929,271.159251,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-14 04:50:00,2008-01-14 04:59:00
4,nbtrace,2008-01-19 08:02:00,2008-01-19 08:19:00,"[DataSource(identifier=51, program='themis-asi...",318.807754,390.216280,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-19 08:05:00,2008-01-19 08:19:00
2,nbtrace,2008-01-19 08:04:00,2008-01-19 08:09:00,"[DataSource(identifier=46, program='themis-asi...",393.493000,494.091856,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-19 08:09:00,2008-01-19 08:04:00
7,nbtrace,2008-01-25 06:40:00,2008-01-25 06:49:00,"[DataSource(identifier=54, program='themis-asi...",327.930714,372.530865,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2008-01-25 06:40:00,2008-01-25 06:49:00


With the search now done and saved to the 's' variable, we can use the Swarm-Aurora helper functions to further examine the results.

NOTE: some IDEs can sometimes have issues figuring out the browser to use when opening Swarm-Aurora. To help with this, you can manually specify the 'browser' parameter to the helper function with one of the available values specified in the official `webbrowser` [module documentation](https://docs.python.org/3/library/webbrowser.html#webbrowser.get).

In [4]:
# open browser to Swarm-Aurora for this search
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s)

# if you want to force it to use Google Chrome, you can do the following
#
# aurorax.search.conjunctions.swarmaurora.open_in_browser(s, browser="google-chrome")

# alternatively, you can also just get the web URL for Swarm-Aurora for this
# specific request, and copy paste it into the browser manually.
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=f63c9bda-985c-4fc6-8ebd-bb8e21c3a756


## UCalgary cloud model

UCalgary has developed a machine learning model for identifying cloud in the THEMIS all-sky imagers. This is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where this particular model thinks there is cloud in the field-of-view at Gillam, over a 7 day period.

More information about this model can be found [here](https://docs.aurorax.space/ml/models/ucalgary_cloud/).

In [5]:
# set timeframe, distance, and conjunction type
start = datetime.datetime(2020, 1, 1, 0, 0, 0)
end = datetime.datetime(2020, 1, 15, 23, 59, 59)
conjunction_types = [aurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# set ground criteria block
ground = [
    aurorax.search.GroundCriteriaBlock(
        programs=["themis-asi"],
        metadata_filters=aurorax.search.MetadataFilter(expressions=[
            # only find records that were classified as not cloud
            aurorax.search.MetadataFilterExpression("calgary_cloud_ml_v1", "classified as not cloudy", operator="="),

            # with a confidence of at least 75%
            aurorax.search.MetadataFilterExpression("calgary_cloud_ml_v1_confidence", 75, operator=">=")
        ]))
]

# set space criteria block
space = [aurorax.search.SpaceCriteriaBlock(programs=["swarm", "elfin"], hemisphere=["northern"])]

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground,
    space=space,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2025-02-27 08:59:06.298941] Search object created
[2025-02-27 08:59:06.317931] Request submitted
[2025-02-27 08:59:06.317956] Request ID: b7a58066-6f5d-4274-9c1d-7a7b057f113e
[2025-02-27 08:59:06.317961] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/b7a58066-6f5d-4274-9c1d-7a7b057f113e
[2025-02-27 08:59:06.317964] Waiting for data ...
[2025-02-27 08:59:07.751963] Checking for data ...
[2025-02-27 08:59:09.185410] Checking for data ...
[2025-02-27 08:59:09.619423] Data is now available
[2025-02-27 08:59:09.619510] Retrieving data ...
[2025-02-27 08:59:10.275355] Retrieved 2.2 MB of data containing 270 records


In [6]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
49,nbtrace,2020-01-01 00:08:00,2020-01-01 00:09:00,"[DataSource(identifier=53, program='themis-asi...",192.029390,336.631805,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 00:09:00,2020-01-01 00:08:00
152,nbtrace,2020-01-01 00:08:00,2020-01-01 00:09:00,"[DataSource(identifier=53, program='themis-asi...",206.588207,260.819127,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 00:09:00,2020-01-01 00:08:00
90,nbtrace,2020-01-01 01:45:00,2020-01-01 01:46:00,"[DataSource(identifier=44, program='themis-asi...",205.885002,420.474697,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 01:46:00,2020-01-01 01:45:00
3,nbtrace,2020-01-01 01:46:00,2020-01-01 01:47:00,"[DataSource(identifier=44, program='themis-asi...",284.420850,493.197476,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 01:46:00,2020-01-01 01:47:00
115,nbtrace,2020-01-01 03:15:00,2020-01-01 03:16:00,"[DataSource(identifier=47, program='themis-asi...",385.355069,414.231784,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 03:16:00,2020-01-01 03:15:00
22,nbtrace,2020-01-01 03:15:00,2020-01-01 03:16:00,"[DataSource(identifier=47, program='themis-asi...",412.812856,489.362899,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 03:16:00,2020-01-01 03:15:00
256,nbtrace,2020-01-01 04:07:00,2020-01-01 04:08:00,"[DataSource(identifier=53, program='themis-asi...",343.134412,396.842659,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 04:07:00,2020-01-01 04:08:00
184,nbtrace,2020-01-01 04:49:00,2020-01-01 04:50:00,"[DataSource(identifier=65, program='themis-asi...",244.886114,355.591219,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 04:49:00,2020-01-01 04:50:00
70,nbtrace,2020-01-01 04:49:00,2020-01-01 04:50:00,"[DataSource(identifier=65, program='themis-asi...",323.719101,352.655457,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 04:49:00,2020-01-01 04:50:00
214,nbtrace,2020-01-01 05:18:00,2020-01-01 05:19:00,"[DataSource(identifier=53, program='themis-asi...",371.020465,478.895130,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2020-01-01 05:19:00,2020-01-01 05:18:00


In [7]:
# have a look at the results in Swarm-Aurora
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=b7a58066-6f5d-4274-9c1d-7a7b057f113e


## Oslo Aurora THEMIS (OATH) model

The OATH model was developed by [Clausen & Nickisch](https://doi.org/10.1029/2018JA025274), and has been run by the AuroraX team for all THEMIS ASI data in the AuroraX search engine.

- Paper: https://doi.org/10.1029/2018JA025274
- More information: http://tid.uio.no/plasma/oath

This paper has two versions of the model, one that is a binary classifier (aurora/no aurora), and one that is a 6-class classifier. AuroraX currently only contains metadata derived from the 6-class model.

The OATH mode is a binary classification performed on a 10-minute basis, and included in all THEMIS ASI AuroraX search engine 'ephemeris' records as a metadata field.

Below, we're going to show an example of finding all 1-minute ephemeris records for any THEMIS ASI instrument where the OATH model has classified the minute as diffuse or discrete aurora, over a 1 day period.

More information about the AuroraX utilization of this model can be found [here](https://docs.aurorax.space/ml/models/clausen_oath/).

In [8]:
# set timeframe, distance, and conjunction types
start = datetime.datetime(2015, 1, 1, 0, 0, 0)
end = datetime.datetime(2015, 1, 31, 23, 59, 59)
conjunction_types = [pyaurorax.search.CONJUNCTION_TYPE_NBTRACE]
distance = 500

# set ground criteria block
ground = [
    aurorax.search.GroundCriteriaBlock(
        programs=["themis-asi"],
        metadata_filters=aurorax.search.MetadataFilter(expressions=[
            aurorax.search.MetadataFilterExpression(
                "clausen_ml_oath",
                [
                    # only find records that have one of these three classifications
                    "classified as diffuse",
                    "classified as discrete",
                    "classified as arc"
                ],
                operator="in")
        ]))
]

# set space criteria block
space = [aurorax.search.SpaceCriteriaBlock(programs=["dmsp"], hemisphere=["northern"])]

# perform the search
s = aurorax.search.conjunctions.search(
    start=start,
    end=end,
    distance=distance,
    ground=ground,
    space=space,
    conjunction_types=conjunction_types,
    verbose=True,
)

[2025-02-27 08:59:10.300165] Search object created
[2025-02-27 08:59:10.321225] Request submitted
[2025-02-27 08:59:10.321252] Request ID: b5af4026-8eee-4f27-8ff6-25ba92193d7e
[2025-02-27 08:59:10.321256] Request details available at: https://api.aurorax.space/api/v1/conjunctions/requests/b5af4026-8eee-4f27-8ff6-25ba92193d7e
[2025-02-27 08:59:10.321259] Waiting for data ...
[2025-02-27 08:59:11.755511] Checking for data ...
[2025-02-27 08:59:12.184118] Data is now available
[2025-02-27 08:59:12.184278] Retrieving data ...
[2025-02-27 08:59:12.803806] Retrieved 1.5 MB of data containing 222 records


In [9]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("start")[0:10]

,conjunction_type,start,end,data_sources,min_distance,max_distance,events,closest_epoch,farthest_epoch
178,nbtrace,2015-01-01 06:38:00,2015-01-01 06:40:00,"[DataSource(identifier=71, program='themis-asi...",129.676030,477.204038,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01 06:39:00,2015-01-01 06:38:00
99,nbtrace,2015-01-01 10:02:00,2015-01-01 10:02:00,"[DataSource(identifier=51, program='themis-asi...",38.161652,38.161652,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01 10:02:00,2015-01-01 10:02:00
87,nbtrace,2015-01-01 10:58:00,2015-01-01 10:58:00,"[DataSource(identifier=48, program='themis-asi...",424.850074,424.850074,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01 10:58:00,2015-01-01 10:58:00
143,nbtrace,2015-01-01 13:36:00,2015-01-01 13:36:00,"[DataSource(identifier=53, program='themis-asi...",477.845592,477.845592,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01 13:36:00,2015-01-01 13:36:00
60,nbtrace,2015-01-01 14:18:00,2015-01-01 14:19:00,"[DataSource(identifier=45, program='themis-asi...",429.648698,469.288619,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01 14:19:00,2015-01-01 14:18:00
148,nbtrace,2015-01-01 16:00:00,2015-01-01 16:00:00,"[DataSource(identifier=57, program='themis-asi...",352.260006,352.260006,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01 16:00:00,2015-01-01 16:00:00
220,nbtrace,2015-01-01 20:10:00,2015-01-01 20:11:00,"[DataSource(identifier=71, program='themis-asi...",285.473795,299.878087,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01 20:10:00,2015-01-01 20:11:00
35,nbtrace,2015-01-01 21:31:00,2015-01-01 21:33:00,"[DataSource(identifier=71, program='themis-asi...",75.318471,425.663394,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-01 21:32:00,2015-01-01 21:33:00
175,nbtrace,2015-01-02 06:23:00,2015-01-02 06:24:00,"[DataSource(identifier=71, program='themis-asi...",335.819298,365.536067,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-02 06:23:00,2015-01-02 06:24:00
196,nbtrace,2015-01-02 09:00:00,2015-01-02 09:00:00,"[DataSource(identifier=71, program='themis-asi...",356.122232,356.122232,"[{'conjunction_type': 'nbtrace', 'e1_source': ...",2015-01-02 09:00:00,2015-01-02 09:00:00


In [10]:
# have a look at the results in Swarm-Aurora
print(aurorax.search.conjunctions.swarmaurora.get_url(s))

https://swarm-aurora.com/conjunctionFinder?aurorax_request_id=b5af4026-8eee-4f27-8ff6-25ba92193d7e


# Ephemeris searching

Let's next have a look at doing ephemeris searches and leverage the ML-derived metadata.

## UCalgary Amorphous Pulsating Aurora (APA) model

In [11]:
# set timeframe and program values
start = datetime.datetime(2008, 1, 5, 0, 0, 0)
end = datetime.datetime(2008, 1, 5, 23, 59, 59)
programs = ["themis-asi"]

# to filter based the ML model data, we need to perform the ephemeris
# search using metadata filters.
#
# below, we're going to filter to only retrieve ephemeris data for when
# the 'calgary_apa_ml_v1' field says 'classified as APA', and has a
# confidence score greater than or equal to 95%.
metadata_filters = aurorax.search.MetadataFilter(expressions=[
    aurorax.search.MetadataFilterExpression("calgary_apa_ml_v1", "classified as APA", operator="="),
    aurorax.search.MetadataFilterExpression("calgary_apa_ml_v1_confidence", 95, operator=">=")
])

# perform the search
s = aurorax.search.ephemeris.search(
    start=start,
    end=end,
    programs=programs,
    metadata_filters=metadata_filters,
    verbose=True,
)

[2025-02-27 08:59:12.838798] Search object created
[2025-02-27 08:59:12.860020] Request submitted
[2025-02-27 08:59:12.860046] Request ID: b1d0647e-44b2-457f-90c5-7742c36db83e
[2025-02-27 08:59:12.860050] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/b1d0647e-44b2-457f-90c5-7742c36db83e
[2025-02-27 08:59:12.860052] Waiting for data ...
[2025-02-27 08:59:14.279900] Checking for data ...
[2025-02-27 08:59:14.711657] Data is now available
[2025-02-27 08:59:14.711747] Retrieving data ...
[2025-02-27 08:59:14.860803] Retrieved 793.8 kB of data containing 230 records


In [12]:
# show the first 10 conjunction results
conjunctions = [d.__dict__ for d in s.data]
df = pd.DataFrame(conjunctions)
df.sort_values("epoch")[0:10]

,data_source,epoch,location_geo,nbtrace,sbtrace,location_gsm,metadata
0,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:10:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.059127724199, lon=-67.1240831...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as diffuse', '..."
1,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:11:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912749361097, lon=-67.12408...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as no aurora',..."
2,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:12:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912726302303, lon=-67.12408...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as no aurora',..."
3,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:13:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912703243496, lon=-67.12408...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as no aurora',..."
4,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:14:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912680184706, lon=-67.12408...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as diffuse', '..."
5,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:15:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912657125913, lon=-67.12408...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as diffuse', '..."
6,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:16:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912634067106, lon=-67.12407...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as diffuse', '..."
7,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:17:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912611008307, lon=-67.12407...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as no aurora',..."
8,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:18:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912587949504, lon=-67.12407...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as no aurora',..."
9,"DataSource(identifier=49, program='themis-asi'...",2008-01-05 07:19:00,"Location(lat=58.109171, lon=-68.411414)","Location(lat=58.109171, lon=-68.411414)","Location(lat=-82.05912564890714, lon=-67.12407...","Location(lat=None, lon=None)","{'clausen_ml_oath': 'classified as no aurora',..."


In [13]:
# print the results of the first 10 ephemeris records as a table showing just
# the classification and confidence score
#
# NOTE: you can use Pandas here like earlier, but here's an example of looking
# at the results without it
print(f"{'Timestamp':<{25}}"
      f"{'calgary_apa_ml_v1':<{25}} "
      f"{'confidence':<{18}}")
print("====================================================================")
for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{25}}"
          f"{s.data[i].metadata['calgary_apa_ml_v1']:<{23}} "  # type: ignore
          f"  {s.data[i].metadata['calgary_apa_ml_v1_confidence']:<{18}} ")  # type: ignore

Timestamp                calgary_apa_ml_v1         confidence        
2008-01-05 07:10:00      classified as APA         99.59              
2008-01-05 07:11:00      classified as APA         99.59              
2008-01-05 07:12:00      classified as APA         99.59              
2008-01-05 07:13:00      classified as APA         99.59              
2008-01-05 07:14:00      classified as APA         99.59              
2008-01-05 07:15:00      classified as APA         99.59              
2008-01-05 07:16:00      classified as APA         99.59              
2008-01-05 07:17:00      classified as APA         99.59              
2008-01-05 07:18:00      classified as APA         99.59              
2008-01-05 07:19:00      classified as APA         99.59              


## UCalgary cloud model

In [14]:
# set up search parameters
start = datetime.datetime(2008, 1, 2, 0, 0, 0)
end = datetime.datetime(2008, 1, 2, 23, 59, 59)
programs = ["themis-asi"]
platforms = ["gillam"]

# we're going to filter to only retrieve ephemeris data for when
# the 'calgary_cloud_ml_v1' field says 'classified as not cloudy',
# and has a confidence score greater than or equal to 75%.
metadata_filters = aurorax.search.MetadataFilter(expressions=[
    aurorax.search.MetadataFilterExpression("calgary_cloud_ml_v1", "classified as not cloudy", operator="="),
    aurorax.search.MetadataFilterExpression("calgary_cloud_ml_v1_confidence", 75, operator=">=")
])

# Perform the search
s = aurorax.search.ephemeris.search(
    start=start,
    end=end,
    programs=programs,
    platforms=platforms,
    metadata_filters=metadata_filters,
    verbose=True,
)

[2025-02-27 08:59:14.880453] Search object created
[2025-02-27 08:59:14.903246] Request submitted
[2025-02-27 08:59:14.903275] Request ID: fc65bdeb-80ba-4c58-8c44-7900a331fd9e
[2025-02-27 08:59:14.903279] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/fc65bdeb-80ba-4c58-8c44-7900a331fd9e
[2025-02-27 08:59:14.903282] Waiting for data ...
[2025-02-27 08:59:16.326946] Checking for data ...
[2025-02-27 08:59:16.747536] Data is now available
[2025-02-27 08:59:16.747618] Retrieving data ...
[2025-02-27 08:59:17.028399] Retrieved 2.1 MB of data containing 593 records


In [15]:
# Again, print some results
print(f"{'Timestamp':<{25}}"
      f"{'Classification':<{30}} "
      f"{'Confidence':<{18}}")
print("====================================================================")
for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{25}}"
          f"{s.data[i].metadata['calgary_cloud_ml_v1']:<{28}} "  # type: ignore
          f"  {s.data[i].metadata['calgary_cloud_ml_v1_confidence']:<{18}} ")  # type: ignore

Timestamp                Classification                 Confidence        
2008-01-02 00:00:00      classified as not cloudy       99.96              
2008-01-02 00:01:00      classified as not cloudy       99.96              
2008-01-02 00:02:00      classified as not cloudy       99.96              
2008-01-02 00:03:00      classified as not cloudy       99.96              
2008-01-02 00:04:00      classified as not cloudy       99.96              
2008-01-02 00:05:00      classified as not cloudy       99.96              
2008-01-02 00:06:00      classified as not cloudy       99.96              
2008-01-02 00:07:00      classified as not cloudy       99.96              
2008-01-02 00:08:00      classified as not cloudy       99.96              
2008-01-02 00:09:00      classified as not cloudy       99.96              


## Oslo Aurora THEMIS (OATH) model

In [ ]:
# set up search parameters
start = datetime.datetime(2008, 1, 1, 6, 0, 0)
end = datetime.datetime(2008, 1, 1, 6, 59, 59)
programs = ["themis-asi"]

# we're going to filter to only retrieve ephemeris data for when
# the 'clausen_ml_oath' field says 'classified as diffuse' or
# 'classified as discrete'
metadata_filters = aurorax.search.MetadataFilter(expressions=[
    aurorax.search.MetadataFilterExpression(
        "clausen_ml_oath",
        ["classified as diffuse", "classified as discrete", "classified as arc"],
        operator="in",
    )
])

# Perform the search
s = aurorax.search.ephemeris.search(
    start=start,
    end=end,
    programs=programs,
    metadata_filters=metadata_filters,
    verbose=True,
)

[2025-02-27 08:59:17.038338] Search object created
[2025-02-27 08:59:17.058938] Request submitted
[2025-02-27 08:59:17.058967] Request ID: f8faf384-0d7f-40b0-9924-2dce18579a03
[2025-02-27 08:59:17.058970] Request details available at: https://api.aurorax.space/api/v1/ephemeris/requests/f8faf384-0d7f-40b0-9924-2dce18579a03
[2025-02-27 08:59:17.058973] Waiting for data ...
[2025-02-27 08:59:18.473203] Checking for data ...
[2025-02-27 08:59:18.894490] Data is now available
[2025-02-27 08:59:18.894577] Retrieving data ...
[2025-02-27 08:59:19.049728] Retrieved 666.8 kB of data containing 193 records


In [17]:
# Again, print some results
print(f"{'Timestamp':<{25}}"
      f"{'Classification':<{18}}")
print("=================================================")

for i in range(10):
    timestamp_str = s.data[i].epoch.strftime('%Y-%m-%d %H:%M:%S')
    print(f"{timestamp_str:<{23}}"
          f"  {s.data[i].metadata['clausen_ml_oath']:<{18}} ")  # type: ignore

Timestamp                Classification    
2008-01-01 06:27:00      classified as arc  
2008-01-01 06:37:00      classified as arc  
2008-01-01 06:38:00      classified as arc  
2008-01-01 06:40:00      classified as arc  
2008-01-01 06:43:00      classified as arc  
2008-01-01 06:45:00      classified as arc  
2008-01-01 06:46:00      classified as arc  
2008-01-01 06:47:00      classified as arc  
2008-01-01 06:49:00      classified as arc  
2008-01-01 06:50:00      classified as arc  
